In [3]:
import numpy as np
import csv
import random
from scipy.sparse import csr_matrix

class Logres:
    """This class is a Logistic regression classifier, which reads data and calculates the weight matrix for it and 
    predicts the output"""
    
    def __init__(self):
        
        self.length = 61189
        self.w = np.empty([20,61189])
        self.data  = np.empty([15000,61189])
        self.n = 0.01
        self.l = 0.01

    def readData(self,location):
        data  = np.zeros([15000,61189]) #to store data, creates a big array to fit all
        length = self.length
        c = 0
        #reads data line by line and stores it in numpy matrix
        with open(location, 'r') as f:
            reader = csv.reader(f)
            count = 1
            for row in reader:
                newData = row[1:length+1]
                data[c] = newData         # fills each row with new data
                c+=1
                count = count+1
                if (count % 1000 == 0):
                    print("looping")
            print ("done")
        f.close()
        data = data[0:(c+1)] #removes unfilled rows
        return data

    def setData(self,data):
        ###perfroms changes of data to match the equation 
        length = self.length
        self.realY = data[:,length-1]   #label values
        data = np.delete(data,length-1,axis=1)  #removes the last column
        data = np.hstack((np.ones((data.shape[0],1)),data)) #adds i in the first column
        
        #creates delta matrix 
        self.y = np.array([[1 if (x1==x2+1) else 0 for x1 in self.realY] for x2 in range(20)])
        self.data = data

    def logres(self):
        w = self.w
        self.bestw = w
        bestloss = 10e10
        s = csr_matrix(self.data)  #sparse matrix for fast multiplication
        y = self.y
        l = self.l
        n = self.n

        count = 0
        for i in range (500):
            previousw = w
            
            #calculate the probability matrix
            pxy1 = np.exp(np.divide((w*s.transpose()),10000))
            
            #replacing last row with one and normalizing
            pxy1[:,pxy1.shape[1]-1] = np.ones([1,20])
            pxy1 = pxy1/pxy1.sum(axis=0,keepdims=1)
            
            #difference between delta and probability
            diff1 = np.subtract(y,pxy1)
            
            #regularization factor
            lw1 = np.multiply(l,w)
            ypx1 = diff1*s
            final = np.subtract(ypx1,lw1)
            
            #updating weight
            w = np.add(w,np.multiply(n,final))
            self.w = w
            count = count+1
            loss = abs(np.absolute(np.subtract(np.argmax(s*w.transpose(),axis=1)+1,self.realY)).sum())
            loss2 = np.max(np.abs(np.subtract(previousw,w)))
            if (loss<bestloss):
                bestw = w
            if (loss<1e-4):
                break
            if (count % 10 == 0):
                print(loss)
                print(loss2)
                print("looping")
        print ("done")
        self.w = w

    def predict(self,newdata):
        ### predicts the label of the given data with respect to current weight matrix
        e1 = np.matmul(self.w,newdata.transpose())
        return np.argmax(e1)+1

    def predictTofile(self,location):
        ### writes the prediction as a csv file
        print ("predicting")
        temp = np.empty([1,61189])
        with open(location,'r') as csvFileObject:
            prediction = [['id','class']]
            csvReader = csv.reader(csvFileObject)
            count = 0
            #loops through each line and calls pred
            for line in csvReader:
                id = line[0]
                line[0]=1
                temp[0] = line
                prediction.append([id,self.predict(temp)])
            print ("writing")
            predictionFile = open('predict.csv','w')
            with predictionFile:
                writer = csv.writer(predictionFile)
                writer.writerows(prediction)

    def test(self,trainData,l,n,testData):
        ###created to test the classifier for different values of variables
        ###given an input and testData, this method returns real labels and predicted labels
        
        #sets the given data
        self.setData(trainData)
        
        #sets the parameters
        self.setln(l,n)
        
        #starts regression
        self.logres()
        
        #gets prediction
        return self.predictMatrix(testData)

    def setln(self,l,n):
        self.l = l
        self.n = n

    def predictMatrix(self, testData):
        ###uses the predict() fucntion to predict the labels of all data in testData
        output = []
        y = testData[:,testData.shape[1]-1]
        testData = np.delete(testData,testData.shape[0],axis=1)
        testData = np.hstack((np.ones((testData.shape[0],1)),testData))
        temp = np.empty([1,61189])
        
        ##loops through data and gets prediction
        for i in range(testData.shape[0]):
            temp[0] = testData[i]
            output.append(self.predict(temp))
        return y,output
        

In [5]:
lg = Logres()
trainlocation = "data/training.csv"
testLocation = "data/testing.csv"

data = lg.readData(trainlocation)



looping
looping
looping
looping
looping
looping
looping
looping
looping
looping
looping
looping
done


In [6]:
lg = Logres()


In [30]:
y,output = lg.test(data[0:9000],0.05,1,data[9000:12000])

In [31]:
def equals (a):
    if (a[0]==a[1]):
        return 1
    else:
        return 0

In [32]:
#calcualtes accuracy from the real and predicted values
sum(list(map(equals,zip(y,output))))/len(y)

0.554

In [5]:
def getPrediction(trainlocation = "data/training.csv",testLocation = "data/testing.csv"):
    ### runs the data writes the prediction of testing in a csv file
    lg = Logres()
    data = lg.readData(trainlocation)
    lg.setData(data)
    lg.logres()
    lg.predictTofile(testLocation)

In [6]:
getPrediction()

looping
looping
looping
looping
looping
looping
looping
looping
looping
looping
looping
looping
done


MemoryError: 